In [ ]:
# 1. Установка зависимостей
!pip install -q ultralytics torch torchvision kagglehub

In [ ]:
# 2. Скачиваем датасет через kagglehub
import kagglehub, os, random, shutil

BASE = kagglehub.dataset_download("gunarakulangr/wild-elephant-yolo-format-dataset")
DS_ROOT   = os.path.join(BASE, "elephant-dataset-yolov")
TRAIN_IMG = os.path.join(DS_ROOT, "train", "images")
TRAIN_LBL = os.path.join(DS_ROOT, "train", "labels")
VAL_IMG   = os.path.join(DS_ROOT, "valid", "images")
VAL_LBL   = os.path.join(DS_ROOT, "valid", "labels")
TEST_IMG  = os.path.join(DS_ROOT, "test",  "images")
TEST_LBL  = os.path.join(DS_ROOT, "test",  "labels")

print(f"Orig sizes → train: {len(os.listdir(TRAIN_IMG))}, val: {len(os.listdir(VAL_IMG))}, test: {len(os.listdir(TEST_IMG))}")

In [ ]:
# 3. Создаём новую маленькую выборку
SAMPLED_ROOT = "/content/elephant_subset"
for split in ("train","valid","test"):
    for sub in ("images","labels"):
        os.makedirs(os.path.join(SAMPLED_ROOT, split, sub), exist_ok=True)

In [ ]:
# 3.1 Сэмплируем 500 train
all_train = sorted(os.listdir(TRAIN_IMG))
random.seed(42)
sample_train = random.sample(all_train, 500)
for fn in sample_train:
    shutil.copy(os.path.join(TRAIN_IMG, fn), os.path.join(SAMPLED_ROOT, "train", "images", fn))
    lbl = fn.replace(".jpg", ".txt")
    shutil.copy(os.path.join(TRAIN_LBL, lbl), os.path.join(SAMPLED_ROOT, "train", "labels", lbl))

In [ ]:
# 3.2 Пропорционально для val и test
orig_n = len(all_train)
val_ratio  = len(os.listdir(VAL_IMG))  / orig_n
test_ratio = len(os.listdir(TEST_IMG)) / orig_n
n_val  = max(1, int(round(500 * val_ratio)))
n_test = max(1, int(round(500 * test_ratio)))

all_val  = sorted(os.listdir(VAL_IMG))
all_test = sorted(os.listdir(TEST_IMG))
sample_val  = random.sample(all_val,  n_val)
sample_test = random.sample(all_test, n_test)

for fn in sample_val:
    shutil.copy(os.path.join(VAL_IMG, fn), os.path.join(SAMPLED_ROOT, "valid", "images", fn))
    lbl = fn.replace(".jpg", ".txt")
    shutil.copy(os.path.join(VAL_LBL, lbl), os.path.join(SAMPLED_ROOT, "valid", "labels", lbl))

for fn in sample_test:
    shutil.copy(os.path.join(TEST_IMG, fn), os.path.join(SAMPLED_ROOT, "test", "images", fn))
    lbl = fn.replace(".jpg", ".txt")
    shutil.copy(os.path.join(TEST_LBL, lbl), os.path.join(SAMPLED_ROOT, "test", "labels", lbl))

print(f"Subsampled → train: {len(sample_train)}, valid: {len(sample_val)}, test: {len(sample_test)}")

In [ ]:
# 4. Генерируем новый data_config.yaml
DATA_YAML = "/content/data_config.yaml"
with open(DATA_YAML, "w") as f:
    f.write(f"""\
path: {SAMPLED_ROOT}
train: train/images
val:   valid/images

names:
  0: elephant
""")
print(open(DATA_YAML).read())

In [ ]:
# 5. Импорт и проверка GPU
from ultralytics import YOLO
import torch
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available(), "| Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


## Тренировка v8n

In [ ]:
from ultralytics import YOLO
import os
import torch

# Проверка доступности GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Пути к данным
MODEL_NAME = "yolov8n.pt"
OUTPUT_DIR = "/kaggle/working/runs"

# Создание модели
model = YOLO(MODEL_NAME)

# Настройки обучения
config = {
    "data": DATA_YAML,
    "epochs": 50,
    "imgsz": 640,
    "batch": 8,
    "device": 0,
    "name": "bicycle_v1",
    "optimizer": "Adam",
    "lr0": 0.001,
    "augment": True,
    "patience": 10,
    "save_period": 5,
    "exist_ok": True,
    "project": OUTPUT_DIR
}

# Запуск обучения
results = model.train(**config)

In [ ]:
from IPython.display import Image, display

model.save('yolo8n_1.pt')

for img in os.listdir(TEST_IMG)[:10]:
  results = model.predict(TEST_IMG + "/" + img, conf=0.5)
  results[0].save("result.jpg")
  display(Image("result.jpg"))

## Тренировка 11n

In [ ]:
from ultralytics import YOLO
import os
import torch

# Проверка доступности GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Пути к данным
MODEL_NAME = "yolo11n.pt"
OUTPUT_DIR = "/kaggle/working/runs"

# Создание модели
model = YOLO(MODEL_NAME)

# Настройки обучения
config = {
    "data": DATA_YAML,
    "epochs": 50,
    "imgsz": 640,
    "batch": 8,
    "device": 0,
    "name": "bicycle_v1",
    "optimizer": "Adam",
    "lr0": 0.001,
    "augment": True,
    "patience": 10,
    "save_period": 5,
    "exist_ok": True,
    "project": OUTPUT_DIR
}

# Запуск обучения
results = model.train(**config)

In [ ]:
from IPython.display import Image, display

model.save('yolo11n_1.pt')

for img in os.listdir(TEST_IMG)[:10]:
  results = model.predict(TEST_IMG + "/" + img, conf=0.5)
  results[0].save("result.jpg")
  display(Image("result.jpg"))

## Тренировка v8m

In [ ]:
from ultralytics import YOLO
import os
import torch

# Проверка доступности GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Пути к данным
MODEL_NAME = "yolov8m.pt"
OUTPUT_DIR = "/kaggle/working/runs"

# Создание модели
model = YOLO(MODEL_NAME)

# Настройки обучения
config = {
    "data": DATA_YAML,
    "epochs": 50,
    "imgsz": 640,
    "batch": 8,
    "device": 0,
    "name": "bicycle_v1",
    "optimizer": "Adam",
    "lr0": 0.001,
    "augment": True,
    "patience": 10,
    "save_period": 5,
    "exist_ok": True,
    "project": OUTPUT_DIR
}

# Запуск обучения
results = model.train(**config)

In [ ]:
from IPython.display import Image, display

model.save('yolov8m_1.pt')

for img in os.listdir(TEST_IMG)[:10]:
  results = model.predict(TEST_IMG + "/" + img, conf=0.5)
  results[0].save("result.jpg")
  display(Image("result.jpg"))

## Тренировка 11m

In [ ]:
from ultralytics import YOLO
import os
import torch

# Проверка доступности GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Пути к данным
MODEL_NAME = "yolo11m.pt"
OUTPUT_DIR = "/kaggle/working/runs"

# Создание модели
model = YOLO(MODEL_NAME)

# Настройки обучения
config = {
    "data": DATA_YAML,
    "epochs": 50,
    "imgsz": 640,
    "batch": 8,
    "device": 0,
    "name": "bicycle_v1",
    "optimizer": "Adam",
    "lr0": 0.001,
    "augment": True,
    "patience": 10,
    "save_period": 5,
    "exist_ok": True,
    "project": OUTPUT_DIR
}

# Запуск обучения
results = model.train(**config)

In [ ]:
from IPython.display import Image, display

model.save('yolo11m_1.pt')

for img in os.listdir(TEST_IMG)[:10]:
  results = model.predict(TEST_IMG + "/" + img, conf=0.5)
  results[0].save("result.jpg")
  display(Image("result.jpg"))

In [ ]:
# 10. Сравнение обученных моделей на тестовом датасете

# 10.1 Устанавливаем дополнительные библиотеки
!pip install -q pandas matplotlib

# 10.2 Подготовка test_config.yaml: укажем и train, и val, оба на тестовые изображения
TEST_YAML = "/content/test_config.yaml"
with open(TEST_YAML, "w") as f:
    f.write(f"""\
path: {SAMPLED_ROOT}
train: test/images
val:   test/images

names:
  0: elephant
""")
print("Test config:")
print(open(TEST_YAML).read())


# 10.3 Импорты
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt

# 10.4 Пути к вашим weight‑файлам
model_paths = {
    "yolov8n": "yolo8n_1.pt",
    "yolo11n": "yolo11n_1.pt",
    "yolov8m": "yolov8m_1.pt",
    "yolo11m": "yolo11m_1.pt"
}

In [ ]:
# 10.5 Оцениваем каждую модель на тест‑сплите
metrics = {}
for name, weights in model_paths.items():
    print(f"=== Evaluating {name} ===")
    res = YOLO(weights).val(data=TEST_YAML, batch=8, device="CPU")
    # Берём mAP@0.5 из результатов
    mAP50 = res.results_dict["metrics/mAP50(B)"]
    metrics[name] = mAP50
    print(f"{name}: mAP@0.5 = {mAP50:.3f}\n")

In [ ]:
# 10.6 Собираем в DataFrame и показываем
df = pd.DataFrame.from_dict(metrics, orient="index", columns=["mAP50"])
df.index.name = "model"
print("Сводная таблица mAP@0.5 на тесте:")
display(df)

In [ ]:
# 10.7 Рисуем столбчатый график
plt.figure(figsize=(8,4))
plt.bar(df.index, df["mAP50"])
plt.title("Сравнение моделей на тестовом датасете")
plt.ylabel("mAP@0.5")
plt.ylim(0,1)
for i, v in enumerate(df["mAP50"]):
    plt.text(i, v+0.02, f"{v:.2f}", ha="center")
plt.show()